# Iterative Back Testing

## Loading in the Data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
import time

C:\Users\aidan\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
C:\Users\aidan\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:167: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True,

In [2]:
#connect to client
from ib_insync import *
util.startLoop()
ib = IB()

In [3]:
ib.connect('127.0.0.1', 7496)

<IB connected to 127.0.0.1:7496 clientId=1>

In [4]:
ib.reqMarketDataType(1)

In [5]:
# For Foreign exchange
forex_contract = Forex("EURUSD")
forex_contract

Forex('EURUSD', exchange='IDEALPRO')

In [6]:
stream_data = ib.reqHistoricalData(
    forex_contract,
    endDateTime = '', 
    durationStr = '2 Y', 
    barSizeSetting = '1 hour', 
    whatToShow = 'MIDPOINT',
    useRTH = True
)

data = util.df(stream_data)
data["date"]= data["date"].dt.tz_convert(None)
data = data.set_index('date')
data

,open,high,low,close,volume,average,barCount
date,,,,,,,
2021-06-07 21:15:00,1.21915,1.21915,1.21885,1.21910,-1.0,-1.0,-1
2021-06-07 22:00:00,1.21910,1.21925,1.21905,1.21920,-1.0,-1.0,-1
2021-06-07 23:00:00,1.21920,1.21930,1.21900,1.21905,-1.0,-1.0,-1
2021-06-08 00:00:00,1.21905,1.21945,1.21825,1.21850,-1.0,-1.0,-1
2021-06-08 01:00:00,1.21850,1.21910,1.21840,1.21875,-1.0,-1.0,-1
...,...,...,...,...,...,...,...
2023-06-07 00:00:00,1.06985,1.07045,1.06975,1.07005,-1.0,-1.0,-1
2023-06-07 01:00:00,1.07005,1.07015,1.06930,1.06965,-1.0,-1.0,-1
2023-06-07 02:00:00,1.06965,1.06965,1.06890,1.06930,-1.0,-1.0,-1


In [7]:
stream_data1 = ib.reqHistoricalData(
    forex_contract,
    endDateTime = '', 
    durationStr = '2 Y', 
    barSizeSetting = '1 hour', 
    whatToShow = 'BID',
    useRTH = True
)

data1 = util.df(stream_data1)
data1["date"]= data1["date"].dt.tz_convert(None)
data1 = data1.set_index('date')
data1

,open,high,low,close,volume,average,barCount
date,,,,,,,
2021-06-07 21:15:00,1.21870,1.21915,1.21830,1.21905,-1.0,-1.0,-1
2021-06-07 22:00:00,1.21905,1.21920,1.21900,1.21915,-1.0,-1.0,-1
2021-06-07 23:00:00,1.21915,1.21925,1.21900,1.21905,-1.0,-1.0,-1
2021-06-08 00:00:00,1.21905,1.21940,1.21825,1.21845,-1.0,-1.0,-1
2021-06-08 01:00:00,1.21845,1.21910,1.21840,1.21875,-1.0,-1.0,-1
...,...,...,...,...,...,...,...
2023-06-07 00:00:00,1.06985,1.07045,1.06975,1.07005,-1.0,-1.0,-1
2023-06-07 01:00:00,1.07005,1.07015,1.06930,1.06965,-1.0,-1.0,-1
2023-06-07 02:00:00,1.06965,1.06965,1.06885,1.06930,-1.0,-1.0,-1


In [8]:
stream_data2 = ib.reqHistoricalData(
    forex_contract,
    endDateTime = '', 
    durationStr = '2 Y', 
    barSizeSetting = '1 hour', 
    whatToShow = 'ASK',
    useRTH = True
)

data2 = util.df(stream_data2)
data2["date"]= data2["date"].dt.tz_convert(None)
data2 = data2.set_index('date')
data2

,open,high,low,close,volume,average,barCount
date,,,,,,,
2021-06-07 21:15:00,1.21955,1.21960,1.21900,1.21915,-1.0,-1.0,-1
2021-06-07 22:00:00,1.21915,1.21925,1.21905,1.21920,-1.0,-1.0,-1
2021-06-07 23:00:00,1.21920,1.21930,1.21905,1.21910,-1.0,-1.0,-1
2021-06-08 00:00:00,1.21910,1.21945,1.21825,1.21850,-1.0,-1.0,-1
2021-06-08 01:00:00,1.21850,1.21910,1.21840,1.21875,-1.0,-1.0,-1
...,...,...,...,...,...,...,...
2023-06-07 00:00:00,1.06990,1.07045,1.06975,1.07005,-1.0,-1.0,-1
2023-06-07 01:00:00,1.07005,1.07015,1.06930,1.06965,-1.0,-1.0,-1
2023-06-07 02:00:00,1.06965,1.06970,1.06890,1.06930,-1.0,-1.0,-1


In [9]:
ib.disconnect()

In [10]:
file = data['close'].to_frame()
file['spread'] = (data2['close']- data1['close'])
file['returns'] = np.log(file["close"].div(file["close"].shift(1)))
file.dropna(inplace = True)
file

,close,spread,returns
date,,,
2021-06-07 22:00:00,1.21920,0.00005,0.000082
2021-06-07 23:00:00,1.21905,0.00005,-0.000123
2021-06-08 00:00:00,1.21850,0.00005,-0.000451
2021-06-08 01:00:00,1.21875,0.00000,0.000205
2021-06-08 02:00:00,1.21835,0.00000,-0.000328
...,...,...,...
2023-06-07 00:00:00,1.07005,0.00000,0.000187
2023-06-07 01:00:00,1.06965,0.00000,-0.000374
2023-06-07 02:00:00,1.06930,0.00000,-0.000327


In [11]:
# to check that all spreads are positive numbers
sum(file['spread']<0)

1

## Using a Iterative backtesting model - model

In [12]:
sma_s = 50
sma_l = 200

In [13]:
file['SMA_S'] = file['close'].rolling(sma_s).mean()
file['SMA_L'] = file['close'].rolling(sma_l).mean()
file.dropna(inplace = True)
file

,close,spread,returns,SMA_S,SMA_L
date,,,,,
2021-06-18 05:00:00,1.19050,0.00005,-0.000714,1.199737,1.211333
2021-06-18 06:00:00,1.18910,0.00000,-0.001177,1.199262,1.211183
2021-06-18 07:00:00,1.19155,0.00000,0.002058,1.198833,1.211045
2021-06-18 08:00:00,1.19130,0.00000,-0.000210,1.198400,1.210909
2021-06-18 09:00:00,1.19170,0.00000,0.000336,1.197969,1.210774
...,...,...,...,...,...
2023-06-07 00:00:00,1.07005,0.00000,0.000187,1.070069,1.071460
2023-06-07 01:00:00,1.06965,0.00000,-0.000374,1.070068,1.071444
2023-06-07 02:00:00,1.06930,0.00000,-0.000327,1.070066,1.071430


In [14]:
file1 = file['close'].to_frame()
file1

,close
date,
2021-06-18 05:00:00,1.19050
2021-06-18 06:00:00,1.18910
2021-06-18 07:00:00,1.19155
2021-06-18 08:00:00,1.19130
2021-06-18 09:00:00,1.19170
...,...
2023-06-07 00:00:00,1.07005
2023-06-07 01:00:00,1.06965
2023-06-07 02:00:00,1.06930


In [15]:
# setting initial position
position = 0

In [16]:
for candle in range(len(file)):
    if file['SMA_S'].iloc[candle] > file['SMA_L'].iloc[candle]:
        if position in [0, -1]:
            print("{}: Go Long | Close: {} | Spread: {}".format(file.index[candle].date(), file.close[candle], file.spread[candle]))
            position = 1
    elif file['SMA_S'].iloc[candle] < file['SMA_L'].iloc[candle]:
        if position in [0, 1]:
            print("{}: Go Short | Close: {} | Spread: {}".format(file.index[candle].date(), file.close[candle], file.spread[candle]))
            position = -1

2021-06-18: Go Short | Close: 1.1905 | Spread: 4.999999999988347e-05
2021-06-28: Go Long | Close: 1.1926 | Spread: 0.0
2021-06-29: Go Short | Close: 1.19005 | Spread: 0.0
2021-07-12: Go Long | Close: 1.18675 | Spread: 0.0
2021-07-14: Go Short | Close: 1.17815 | Spread: 5.0000000000105516e-05
2021-07-27: Go Long | Close: 1.1836 | Spread: 5.0000000000105516e-05
2021-08-05: Go Short | Close: 1.18325 | Spread: 4.999999999988347e-05
2021-08-16: Go Long | Close: 1.1777 | Spread: 0.0
2021-08-18: Go Short | Close: 1.17205 | Spread: 5.0000000000105516e-05
2021-08-24: Go Long | Close: 1.17575 | Spread: 0.0
2021-09-09: Go Short | Close: 1.18185 | Spread: 5.0000000000105516e-05
2021-10-14: Go Long | Close: 1.15905 | Spread: 0.0
2021-10-26: Go Short | Close: 1.159 | Spread: 0.0
2021-10-29: Go Long | Close: 1.16705 | Spread: 0.0
2021-11-01: Go Short | Close: 1.1555 | Spread: 0.0
2021-11-10: Go Long | Close: 1.15775 | Spread: 4.999999999988347e-05
2021-11-10: Go Short | Close: 1.1479 | Spread: 4.9999

## Iterative Base Function

In [115]:
class IterativeBase():
    ''' Class for the iterative backtesting of Machine Learning-based trading strategies (Classification).
    '''
    def __init__(self, symbol, start, end, amount, use_spread = True):
        '''
        Parameters
        ----------
        symbol: str
            ticker symbol (instrument) to be backtested
        start: str
            start date for data import
        end: str
            end date for data import
        tc: float
            proportional transaction/trading costs per trade
        '''
        self.symbol = symbol
        self.start = start
        self.end = end
        self.initial_balance = amount
        self.current_balance = amount
        self.units = 0
        self.trades = 0
        self.position = 0
        self.use_spread = use_spread
        self.get_data()
                             
    def get_data(self):
        ''' Imports the data (source can be changed).
        '''
        raw = file1.copy()
        raw = raw.loc[self.start:self.end]
        raw["returns"] = np.log(raw.close / raw.close.shift(1))
        raw['spread'] = file['spread']
        raw = raw.dropna()
        self.data = raw
        
    def plot_data(self, columns = None):
        if columns is None:
            columns = 'close'
        self.data[columns].plot(figsize = (12,8), title = self.symbol)

    def get_values(self, candle):
        date = str(self.data.index[candle].date())
        close = round(self.data.close.iloc[candle], 5)
        spread = round(self.data.spread.iloc[candle], 5)
        return date, close, spread

    def print_current_balance(self, candle):
        date, close, spread = self.get_values(candle)
        print("{} | Current Balance: {}".format(date, round(self.current_balance, 2)))
    
    def buy_instrument(self, candle, units = None, amount = None):
        date, close, spread = self.get_values(candle)
        if self.use_spread:
            close += spread/2 # ask price
        if amount is not None: #use units if units are passed, otherwise calculate units
            units = int(amount / close)
        self.current_balance -= units * close # reduce cash balance by "purchase price"
        self.units += units
        self.trades +=1
        print("{} | Buying {} for {}".format(date, units, round(close,5)))
        
    def sell_instrument(self, candle, units = None, amount = None):
        date, close, spread = self.get_values(candle)
        if self.use_spread:
            close -= spread/2 # bid price
        if amount is not None: #use units if units are passed, otherwise calculate units
            units = int(amount / close)
        self.current_balance += units * close # increase cash balance by "purchase price"
        self.units -= units
        self.trades +=1
        print("{} | Selling {} for {}".format(date, units, round(close,5)))
    
    def print_current_position_value(self, candle):
        date, close, spread = self.get_values(candle)
        cpv = self.units * close
        print("{} | Current Position Value = {}".format(date, round(cpv,5)))
    
    def print_current_nav(self, candle):
        date, close, spread = self.get_values(candle)
        nav = self.current_balance + self.units * close
        print("{} | Net Asset Value = {}".format(date, round(nav,5)))
        
    def close_pos(self, candle):
        date, close, spread = self.get_values(candle)
        print(75 * "-")
        print("{} | +++ Closing Final Position +++".format(date))
        self.current_balance += self.units * close # closing final position (works with short and long!)
        self.current_balance -= (abs(self.units) * spread/2 * self.use_spread) # substract half-spread costs
        print("{} | closing position of {} for {}".format(date, self.units, close))
        self.units = 0 # setting position to neutral
        self.trades += 1
        perf = (self.current_balance - self.initial_balance) / self.initial_balance * 100
        self.print_current_balance(candle)
        print("{} | net performance (%) = {}".format(date, round(perf, 2) ))
        print("{} | number of trades executed = {}".format(date, self.trades))
        print(75 * "-")

In [116]:
model = IterativeBase('EURUSD', '2021-08-03', '2023-06-06', 100000 )
model

In [117]:
model.data

,close,returns,spread
date,,,
2021-08-03 01:00:00,1.18750,0.000253,0.00000
2021-08-03 02:00:00,1.18710,-0.000337,0.00000
2021-08-03 03:00:00,1.18745,0.000295,0.00005
2021-08-03 04:00:00,1.18805,0.000505,0.00005
2021-08-03 05:00:00,1.18750,-0.000463,0.00000
...,...,...,...
2023-06-06 19:00:00,1.06920,-0.000234,0.00000
2023-06-06 20:00:00,1.06925,0.000047,0.00000
2023-06-06 21:15:00,1.06945,0.000187,0.00015


In [118]:
# first candle 
model.get_values(0)

('2021-08-03', 1.1875, 0.0)

In [119]:
#check balance at first candle
model.print_current_balance(0)

2021-08-03 | Current Balance: 100000


In [120]:
#buy instrument at first candle
model.buy_instrument(0, units = 1000)

2021-08-03 | Buying 1000 for 1.1875


In [121]:
#balance after purchase of 1000 units
model.print_current_balance(0)

2021-08-03 | Current Balance: 98812.5


In [122]:
#current position value
model.print_current_position_value(0)
model.print_current_nav(0)

2021-08-03 | Current Position Value = 1187.5
2021-08-03 | Net Asset Value = 100000.0


In [123]:
#next candle postion value
model.print_current_position_value(1)
model.print_current_nav(1)

2021-08-03 | Current Position Value = 1187.1
2021-08-03 | Net Asset Value = 99999.6


In [124]:
#at the end
model.print_current_position_value(-1)
model.print_current_nav(-1)

2023-06-06 | Current Position Value = 1069.85
2023-06-06 | Net Asset Value = 99882.35


In [125]:
#sell instrument
model.sell_instrument(1, units= 10000)
model.print_current_position_value(1)
model.print_current_nav(1)

2021-08-03 | Selling 10000 for 1.1871
2021-08-03 | Current Position Value = -10683.9
2021-08-03 | Net Asset Value = 99999.6


In [126]:
#close out postions
model.close_pos(-1)

---------------------------------------------------------------------------
2023-06-06 | +++ Closing Final Position +++
2023-06-06 | closing position of -9000 for 1.06985
2023-06-06 | Current Balance: 101054.62
2023-06-06 | net performance (%) = 1.05
2023-06-06 | number of trades executed = 3
---------------------------------------------------------------------------


## Creating a Backtesting function

In [138]:
class IterativeBacktesting(IterativeBase):
    
    # helper method
    def go_long(self, candle, units = None, amount = None):
        if self.position == -1:
            self.buy_instrument(candle, units = -self.units) # if short position, go neutral first
        if units:
            self.buy_instrument(candle, units = units)
        elif amount:
            if amount == "all":
                amount = self.current_balance
            self.buy_instrument(candle, amount = amount) # go long

    # helper method
    def go_short(self, candle, units = None, amount = None):
        if self.position == 1:
            self.sell_instrument(candle, units = self.units) # if long position, go neutral first
        if units:
            self.sell_instrument(candle, units = units)
        elif amount:
            if amount == "all":
                amount = self.current_balance
            self.sell_instrument(candle, amount = amount) # go short

    def test_sma_strategy(self, SMA_S, SMA_L):
        ''' 
        Backtests an SMA crossover strategy with SMA_S (short) and SMA_L (long).
        
        Parameters
        ----------
        SMA_S: int
            moving window in bars (e.g. days) for shorter SMA
        SMA_L: int
            moving window in bars (e.g. days) for longer SMA
        '''
        
        # nice printout
        stm = "Testing SMA strategy | {} | SMA_S = {} & SMA_L = {}".format(self.symbol, SMA_S, SMA_L)
        print("-" * 75)
        print(stm)
        print("-" * 75)
        
        # reset 
        self.position = 0  # initial neutral position
        self.trades = 0  # no trades yet
        self.current_balance = self.initial_balance  # reset initial capital
        self.get_data() # reset dataset
        
        # prepare data
        self.data["SMA_S"] = self.data["close"].rolling(SMA_S).mean()
        self.data["SMA_L"] = self.data["close"].rolling(SMA_L).mean()
        self.data.dropna(inplace = True)

        # sma crossover strategy
        for candle in range(len(self.data)-1): # all bars (except the last bar)
            if self.data["SMA_S"].iloc[candle] > self.data["SMA_L"].iloc[candle]: # signal to go long
                if self.position in [0, -1]:
                    self.go_long(candle, amount = "all") # go long with full amount
                    self.position = 1  # long position
            elif self.data["SMA_S"].iloc[candle] < self.data["SMA_L"].iloc[candle]: # signal to go short
                if self.position in [0, 1]:
                    self.go_short(candle, amount = "all") # go short with full amount
                    self.position = -1 # short position
        self.close_pos(candle+1) # close position at the last bar
        
        
    def test_con_strategy(self, window = 1):
        ''' 
        Backtests a simple contrarian strategy.
        
        Parameters
        ----------
        window: int
            time window (number of bars) to be considered for the strategy.
        '''
        
        # nice printout
        stm = "Testing Contrarian strategy | {} | Window = {}".format(self.symbol, window)
        print("-" * 75)
        print(stm)
        print("-" * 75)
        
        # reset 
        self.position = 0  # initial neutral position
        self.trades = 0  # no trades yet
        self.current_balance = self.initial_balance  # reset initial capital
        self.get_data() # reset dataset
        
        # prepare data
        self.data["rolling_returns"] = self.data["returns"].rolling(window).mean()
        self.data.dropna(inplace = True)
        
        # Contrarian strategy
        for candle in range(len(self.data)-1): # all bars (except the last bar)
            if self.data["rolling_returns"].iloc[candle] <= 0: #signal to go long
                if self.position in [0, -1]:
                    self.go_long(candle, amount = "all") # go long with full amount
                    self.position = 1  # long position
            elif self.data["rolling_returns"].iloc[candle] > 0: #signal to go short
                if self.position in [0, 1]:
                    self.go_short(candle, amount = "all") # go short with full amount
                    self.position = -1 # short position
        self.close_pos(candle+1) # close position at the last bar
        
        
    def test_boll_strategy(self, SMA, dev):
        ''' 
        Backtests a Bollinger Bands mean-reversion strategy.
        
        Parameters
        ----------
        SMA: int
            moving window in bars (e.g. days) for simple moving average.
        dev: int
            distance for Lower/Upper Bands in Standard Deviation units
        '''
        
        # nice printout
        stm = "Testing Bollinger Bands Strategy | {} | SMA = {} & dev = {}".format(self.symbol, SMA, dev)
        print("-" * 75)
        print(stm)
        print("-" * 75)
        
        # reset 
        self.position = 0  # initial neutral position
        self.trades = 0  # no trades yet
        self.current_balance = self.initial_balance  # reset initial capital
        self.get_data() # reset dataset
        
        # prepare data
        self.data["SMA"] = self.data["close"].rolling(SMA).mean()
        self.data["Lower"] = self.data["SMA"] - self.data["close"].rolling(SMA).std() * dev
        self.data["Upper"] = self.data["SMA"] + self.data["close"].rolling(SMA).std() * dev
        self.data.dropna(inplace = True) 
        
        # Bollinger strategy
        for candle in range(len(self.data)-1): # all bars (except the last bar)
            if self.position == 0: # when neutral
                if self.data["close"].iloc[candle] < self.data["Lower"].iloc[candle]: # signal to go long
                    self.go_long(candle, amount = "all") # go long with full amount
                    self.position = 1  # long position
                elif self.data["close"].iloc[candle] > self.data["Upper"].iloc[candle]: # signal to go Short
                    self.go_short(candle, amount = "all") # go short with full amount
                    self.position = -1 # short position
            elif self.position == 1: # when long
                if self.data["close"].iloc[candle] > self.data["SMA"].iloc[candle]:
                    if self.data["close"].iloc[candle] > self.data["Upper"].iloc[candle]: # signal to go short
                        self.go_short(candle, amount = "all") # go short with full amount
                        self.position = -1 # short position
                    else:
                        self.sell_instrument(candle, units = self.units) # go neutral
                        self.position = 0
            elif self.position == -1: # when short
                if self.data["close"].iloc[candle] < self.data["SMA"].iloc[candle]:
                    if self.data["close"].iloc[candle] < self.data["Lower"].iloc[candle]: # signal to go long
                        self.go_long(candle, amount = "all") # go long with full amount
                        self.position = 1 # long position
                    else:
                        self.buy_instrument(candle, units = -self.units) # go neutral
                        self.position = 0                
        self.close_pos(candle+1) # close position at the last bar

In [139]:
model1 = IterativeBacktesting('EURUSD', '2021-08-03', '2023-06-06', 100000, use_spread= True)
model1

In [140]:
model1.test_sma_strategy(50,200)

---------------------------------------------------------------------------
Testing SMA strategy | EURUSD | SMA_S = 50 & SMA_L = 200
---------------------------------------------------------------------------
2021-08-13 | Selling 85162 for 1.17423
2021-08-16 | Buying 85162 for 1.1777
2021-08-16 | Buying 84659 for 1.1777
2021-08-18 | Selling 84659 for 1.17203
2021-08-18 | Selling 84659 for 1.17203
2021-08-24 | Buying 84659 for 1.17575
2021-08-24 | Buying 84123 for 1.17575
2021-09-09 | Selling 84123 for 1.18183
2021-09-09 | Selling 84123 for 1.18183
2021-10-14 | Buying 84123 for 1.15905
2021-10-14 | Buying 87429 for 1.15905
2021-10-26 | Selling 87429 for 1.159
2021-10-26 | Selling 87429 for 1.159
2021-10-29 | Buying 87429 for 1.16705
2021-10-29 | Buying 86223 for 1.16705
2021-11-01 | Selling 86223 for 1.1555
2021-11-01 | Selling 86223 for 1.1555
2021-11-10 | Buying 86223 for 1.15778
2021-11-10 | Buying 85884 for 1.15778
2021-11-10 | Selling 85884 for 1.14788
2021-11-10 | Selling 85884 fo

In [143]:
model1.test_boll_strategy(20, 1)

---------------------------------------------------------------------------
Testing Bollinger Bands Strategy | EURUSD | SMA = 20 & dev = 1
---------------------------------------------------------------------------
2021-08-04 | Selling 84183 for 1.18788
2021-08-04 | Buying 84183 for 1.1851
2021-08-04 | Buying 84578 for 1.1851
2021-08-04 | Selling 84578 for 1.1889
2021-08-04 | Selling 84578 for 1.1889
2021-08-04 | Buying 84578 for 1.1843
2021-08-04 | Buying 85235 for 1.1843
2021-08-05 | Selling 85235 for 1.1849
2021-08-05 | Buying 85355 for 1.18322
2021-08-05 | Selling 85355 for 1.18458
2021-08-05 | Selling 85355 for 1.18458
2021-08-05 | Buying 85355 for 1.1836
2021-08-05 | Buying 85519 for 1.18327
2021-08-09 | Selling 85519 for 1.17615
2021-08-09 | Selling 85509 for 1.1763
2021-08-09 | Buying 85509 for 1.17448
2021-08-09 | Buying 85774 for 1.17448
2021-08-11 | Selling 85774 for 1.1725
2021-08-11 | Buying 85888 for 1.17095
2021-08-11 | Selling 85888 for 1.17368
2021-08-11 | Selling 8588

2022-08-23 | Selling 108456 for 0.99972
2022-08-23 | Selling 108456 for 0.99972
2022-08-24 | Buying 108456 for 0.99495
2022-08-24 | Buying 109535 for 0.9946
2022-08-24 | Selling 109535 for 0.999
2022-08-24 | Selling 109535 for 0.999
2022-08-25 | Buying 109535 for 0.99805
2022-08-25 | Buying 109937 for 0.9963
2022-08-26 | Selling 109937 for 0.99742
2022-08-26 | Selling 109774 for 0.9989
2022-08-26 | Buying 109774 for 0.998
2022-08-28 | Buying 110318 for 0.99488
2022-08-29 | Selling 110318 for 0.99627
2022-08-29 | Selling 110077 for 0.99845
2022-08-30 | Buying 110077 for 0.99912
2022-08-30 | Selling 109515 for 1.0029
2022-08-30 | Buying 109515 for 0.99908
2022-08-30 | Buying 110353 for 0.99908
2022-08-30 | Selling 110353 for 1.0018
2022-08-31 | Selling 110197 for 1.00322
2022-08-31 | Buying 110197 for 1.00157
2022-08-31 | Buying 110714 for 1.00018
2022-08-31 | Selling 110714 for 1.00548
2022-08-31 | Selling 110714 for 1.00548
2022-09-01 | Buying 110714 for 1.00238
2022-09-01 | Buying 111